# Hospital Geolocation Webscrape


In [61]:
import pandas as pd
import requests
import bs4
import re
import json
import geocoder

### UTILITY FUNCTIONS

In [2]:
# A beautiful soup web scraping function
def run_bs4(link, lxml=None):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
    headers = {'User-Agent': user_agent}
    page = requests.get(link)
    html = page.content
    soup = bs4.BeautifulSoup(html, 'html.parser')
    return(soup)

In [ ]:
# a regex function to find hospital names off a wiki link page 
def find_names(list_):
    paragraph = ''
    for x in list_:
        #print(x)
        try:
            paragraph += str(x).replace('%27',"") + ' '
        except:
            pass
    return(re.findall(r'/wiki/([\S]+)', paragraph))

### CODE TO GET HOSPITAL NAMES AND GEOLOCATIONS

In [162]:
# wget the wiki page of hospitals in the Bay area
!wget https://en.wikipedia.org/wiki/Category:Hospitals_in_the_San_Francisco_Bay_Area

In [ ]:
# Run soup and other scrape functions to get a list of hospital names in the Bay Area
soup = ' '
with open('Category:Hospitals_in_the_San_Francisco_Bay_Area', 'r') as data_file:
          for x in data_file.readlines():
            soup += x 

soup_p = bs4.BeautifulSoup(soup)

list_of_hospitals = []
category = []

# gets links to Oakland and SF City hospitals
for x in soup_p.find_all('a'):
    if 'Categories:' in str(x.get('href')):
        category.append(str(x.get('href')))
    if ':' in str(x.get('href')):
        pass
    elif '.org' in str(x.get('href')):
        pass
    elif 'Main' in str(x.get('href')):
        pass
    else:
        list_of_hospitals.append(x.get('href'))

# creates soup links for oakland and SFCity hospitals
hospt_ = []
for x in category[1:3]:
    print(x)
    new_link = 'https://en.wikipedia.org/' + x
    hospt_.append(new_link)
    
# scrapes hospital names off oakland and SF city wiki pages  
other_hospt = []
for link in hospt_:
    soup_p = run_bs4(link)
    for x in soup_p.find_all('a'):
        if ':' in str(x.get('href')):
            pass
        elif '.org' in str(x.get('href')):
            pass
        elif 'Main' in str(x.get('href')):
            pass
        else:
            other_hospt.append(x.get('href'))
            

# Runs regex function to get just the names of hospitals (THE WIKI PAGE WAS A PAGE OF LINKS)
hospital_names = find_names(list_of_hospitals[6:])
other_names = find_names(other_hospt[3:])
names = hospital_names + other_names

NameError: name 'names' is not defined

### GEOCODER FUNCTIONS TO GET LAT AND LONG OF HOSPITALS 

In [66]:
!ls ../data

Alameda_Hospital_Venues.csv
Category:Hospitals_in_the_San_Francisco_Bay_Area
FOOD&HEALTHWorkersBAYAREA.csv
Geospatial_data
Hospital_CLusters_and_venues.csv
Hospital_locations
Hospital_locations.csv
bay_areaApr19.csv
bay_areaApr28.csv
bay_area_geo.cvs
cluster_april_21.csv
cluster_april_29.csv
corrected_addresses.csv
food_health_workers.csv
hospital_venues_ratings.csv
hospital_venues_ratings_april_23.csv
hospitals_sf.html
marin_hospit.csv
postalcodesT


In [67]:
add = pd.read_csv('../data/marin_hospit.csv')

In [68]:
add

,name,lat,lng,address,locale
0,sonoma specialty hospital,38.397241,-122.819054,"501 Petaluma Ave, Sebastopol, CA 95472, USA",Sebastopol
1,novato community hospital,38.099153,-122.560229,"180 Rowland Way, Novato, CA 94945, USA",Novato
2,petaluma valley hospital,38.254591,-122.631576,"400 N McDowell Blvd, Petaluma, CA 94954, USA",Petaluma
3,san rafael convalescent,38.002665,-122.520896,"234 N San Pedro Rd, San Rafael, CA 94903, USA",San Rafael
4,kentfield hospital kentfield,37.958745,-122.552432,"1125 Sir Francis Drake Blvd, Kentfield, CA 949...",Kentfield
5,sonoma valley hospital,38.287975,-122.465300,"347 Andrieux St, Sonoma, CA 95476, USA",Sonoma
6,sutter health novato,38.099152,-122.560230,"165 Rowland Way, Novato, CA 94945, USA",Novato
7,santa rosa memorial,38.443620,-122.701388,"1165 Montgomery Dr, Santa Rosa, CA 95405, USA",Santa Rosa
8,kaiser permanente san,37.768267,-122.395312,"1600 Owens St, San Francisco, CA 94158, USA",San Francisco


In [262]:
print(names[0])
url = 'https://maps.googleapis.com/maps/api/geocode/json?'
params = {'address': names[2] + 'California', 'key' :google_api_key}
r = requests.get(url, params=params)
results = r.json()
print(results['results'][0]['formatted_address'])
hospital_loc[hosp] = {'name' : hosp, 'lat': results['results'][0]['geometry']['location']['lat'], \
                         'lng': results['results'][0]['geometry']['location']['lng']}

Agnews_Developmental_Center
2356 Sutter St, San Francisco, CA 94115, USA


In [269]:
# scrape results off geocode and put into a dictionary of hospitals
hospital_loc = {}
for hosp in names:
    hosp = hosp.replace('_', " ")
    url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    params = {'address': hosp + 'California', 'key' :google_api_key}
    r = requests.get(url, params=params)
    results = r.json()
    for x in results['results'][0]['address_components']:
        if 'locality' in x['types']:
            locale = x['long_name']
    hospital_loc[hosp] = {'name' : hosp, 'lat': results['results'][0]['geometry']['location']['lat'], \
                         'lng': results['results'][0]['geometry']['location']['lng'], \
                          'address': results['results'][0]['formatted_address'], \
                          'locale':locale}

In [270]:
hospital_loc

{'Agnews Developmental Center': {'name': 'Agnews Developmental Center',
  'lat': 37.3946625,
  'lng': -121.9591252,
  'address': 'Agnew, Santa Clara, CA 95054, USA',
  'locale': 'Santa Clara'},
 'Alameda Hospital': {'name': 'Alameda Hospital',
  'lat': 37.762677,
  'lng': -122.2541547,
  'address': '2070 Clinton Ave, Alameda, CA 94501, USA',
  'locale': 'Alameda'},
 'Alcatraz Hospital': {'name': 'Alcatraz Hospital',
  'lat': 37.7854667,
  'lng': -122.4393181,
  'address': '2356 Sutter St, San Francisco, CA 94115, USA',
  'locale': 'San Francisco'},
 'Alta Bates Summit Medical Center': {'name': 'Alta Bates Summit Medical Center',
  'lat': 37.8216476,
  'lng': -122.262967,
  'address': '350 Hawthorne Ave, Oakland, CA 94609, USA',
  'locale': 'Oakland'},
 'Brookside Hospital': {'name': 'Brookside Hospital',
  'lat': 37.9537211,
  'lng': -122.3372782,
  'address': 'Brookside Hospital Heliport, San Pablo, CA 94806, USA',
  'locale': 'San Pablo'},
 'Doctors Medical Center San Pablo Campus': 

In [271]:
### MAKES A CSV OF HOSPITAL NAMES WITH LAT AND LONG 
df_hos = pd.DataFrame(hospital_loc).T
df_hos.to_csv('Hospital_locations.csv', index=False)

# Scrape for Marin County Hospitals

In [3]:
# first url for list of hospitals in marin
marin_soup =run_bs4('https://www.yelp.com/search?cflt=hospitals&find_loc=Marin+County%2C+CA')

In [9]:
# backup irl for hospitals
marin_soup_2 = run_bs4('https://www.marinhhs.org/resources/Emergency/Hospitals')

In [17]:
# find links and names of hospitals using beautiful soup
marin_hosp_2 = []
for x in marin_soup_2.find_all('section'):
    for y in x.find_all('a'):
        link = y.get('href')
        print(link)
        
    

/resources/Emergency/Counseling
/resources/Emergency/Emergency-Shelters
/resources/Emergency/Hospitals
/resources/Emergency/Hospitals?items_per_page=5
/community-resource-guide/marin-county-emergency-services-911
/community-resource-guide/marin-general-hospital
mailto:webmaster@maringeneral.org
http://www.maringeneral.org/
https://maps.google.com/?q=Marin General Hospital Greenbrae 94904
/community-resource-guide/novato-community-hospital
http://www.novatocommunity.sutterhealth.org/
https://maps.google.com/?q=Novato Community Hospital Novato 94945
/community-resource-guide/kaiser-permanente-san-rafael-medical-center
https://thrive.kaiserpermanente.org/care-near-you/northern-california/sanrafael/
https://maps.google.com/?q=Kaiser Permanente: San Rafael Medical Center San Rafael 94903
/community-resource-guide/kentfield-rehabilitation-specialty-hospital-outpatient-center
https://maps.google.com/?q=Kentfield Rehabilitation & Specialty Hospital: Outpatient Center Kentfield 94904
/community

In [8]:
# delete redundant links
list(set(marin_hosp))

['/biz/sonoma-specialty-hospital-sebastopol-2',
 '/biz/novato-community-hospital-novato?hrid=sIMiMqRY_-oD_qy8peM0Tw',
 '/biz/petaluma-valley-hospital-petaluma',
 '/biz/san-rafael-convalescent-hospital-san-rafael',
 '/biz/petaluma-valley-hospital-petaluma?hrid=ThmjeNKe4PYRKTTs6xxRUQ',
 '/biz/kentfield-hospital-kentfield-2?hrid=Rj8k4RfU1nFGzhVOq6W3Fg',
 '/biz/kentfield-hospital-kentfield-2',
 '/biz/novato-community-hospital-novato-3',
 '/biz/sonoma-valley-hospital-sonoma?hrid=pv3O7JHo-J2JjOq5037Xrw',
 '/biz/sutter-health-novato-2',
 '/biz/santa-rosa-memorial-hospital-santa-rosa-2?hrid=PRqC-ACC5XRr5ERMZdaxWQ',
 '/biz/sutter-health-novato-2?hrid=AyJ2qDhvQp_6-XxG2wUCAg',
 '/biz/novato-community-hospital-novato',
 '/biz/san-rafael-convalescent-hospital-san-rafael?hrid=LdieCd61CITh1fdewwaDcg',
 '/biz/sonoma-specialty-hospital-sebastopol-2?hrid=Gowmyp54Sbvu56zzHhP80Q',
 '/biz/sonoma-valley-hospital-sonoma',
 '/biz/kaiser-permanente-san-rafael-medical-center-san-rafael?hrid=DS194kCT8FG3dy4der6I

In [38]:
# regex to find names out of links
import re
marin_hopsital = []
for x in list(set(marin_hosp)):
    
    hospital = re.findall(r'/biz/([a-z]+-[a-z]+([a-z]+)?(-[a-z]+)?)', x)
    name = ''
    for z in hospital[0][0].split('-'):
        name += str(z) + ' '
    marin_hopsital.append(name)

In [51]:
# use Geocode to find addresses and lat and long.
hospital_loc = {}
for hosp in marin_hopsital:
    print(hosp)
    hosp = hosp.replace('_', " ")
    url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    params = {'address': hosp + 'California', 'key' :google_api_key}
    r = requests.get(url, params=params)
    results = r.json()
    for x in results['results'][0]['address_components']:
        if 'locality' in x['types']:
            locale = x['long_name']
    hospital_loc[hosp] = {'name' : hosp, 'lat': results['results'][0]['geometry']['location']['lat'], \
                         'lng': results['results'][0]['geometry']['location']['lng'], \
                          'address': results['results'][0]['formatted_address'], \
                          'locale':locale}
   

sonoma specialty hospital 
novato community hospital 
petaluma valley hospital 
san rafael convalescent 
petaluma valley hospital 
kentfield hospital kentfield 
kentfield hospital kentfield 
novato community hospital 
sonoma valley hospital 
sutter health novato 
santa rosa memorial 
sutter health novato 
novato community hospital 
san rafael convalescent 
sonoma specialty hospital 
sonoma valley hospital 
kaiser permanente san 
kaiser permanente san 
santa rosa memorial 


In [59]:
# make a dataframe and save info to csv
pd.DataFrame(hospital_loc).T.to_csv('../data/marin_hospit.csv', index=False)

In [56]:
# check dictionary
hospital_loc

{'sonoma specialty hospital ': {'name': 'sonoma specialty hospital ',
  'lat': 38.3972414,
  'lng': -122.8190544,
  'address': '501 Petaluma Ave, Sebastopol, CA 95472, USA',
  'locale': 'Sebastopol'},
 'novato community hospital ': {'name': 'novato community hospital ',
  'lat': 38.099153,
  'lng': -122.560229,
  'address': '180 Rowland Way, Novato, CA 94945, USA',
  'locale': 'Novato'},
 'petaluma valley hospital ': {'name': 'petaluma valley hospital ',
  'lat': 38.2545915,
  'lng': -122.6315763,
  'address': '400 N McDowell Blvd, Petaluma, CA 94954, USA',
  'locale': 'Petaluma'},
 'san rafael convalescent ': {'name': 'san rafael convalescent ',
  'lat': 38.0026646,
  'lng': -122.5208956,
  'address': '234 N San Pedro Rd, San Rafael, CA 94903, USA',
  'locale': 'San Rafael'},
 'kentfield hospital kentfield ': {'name': 'kentfield hospital kentfield ',
  'lat': 37.9587453,
  'lng': -122.5524318,
  'address': '1125 Sir Francis Drake Blvd, Kentfield, CA 94904, USA',
  'locale': 'Kentfield